In [1]:
!pip install simpy

In [ ]:
## Bank Renege

In [2]:
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
def source(env, number, interval, counter):
    """Clientes al azar"""
    for i in range(number):
        c = customer(env, 'Cliente%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)
        
def customer(env, name, counter, time_in_bank):
    """El cliente llega, se sirve y se va."""
    arrive = env.now
    print('%7.4f %s: Aqui estoy' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Espere el contador o aborte al final de nuestra atadura.
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # Llegamos al mostrador
            print('%7.4f %s: Esperé %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Salí' % (env.now, name))

        else:
            # Renegar
            print('%7.4f %s: RENIEGO despues de %6.3f' % (env.now, name, wait))


# Setup e inicia simulacion
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# inicia procesos y ejecuta
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()        

Bank renege
 0.0000 Cliente00: Aqui estoy
 0.0000 Cliente00: Esperé  0.000
 3.8595 Cliente00: Salí
10.2006 Cliente01: Aqui estoy
10.2006 Cliente01: Esperé  0.000
12.7265 Cliente02: Aqui estoy
13.9003 Cliente02: RENIEGO despues de  1.174
23.7507 Cliente01: Salí
34.9993 Cliente03: Aqui estoy
34.9993 Cliente03: Esperé  0.000
37.9599 Cliente03: Salí
40.4798 Cliente04: Aqui estoy
40.4798 Cliente04: Esperé  0.000
43.1401 Cliente04: Salí


In [ ]:
##Carwash

In [10]:
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes


class Carwash(object):
    """Un lavado de autos tiene un número limitado de máquinas (` `NUM_MACHINES``) para
     coches limpios en paralelo.

     Los coches tienen que solicitar una de las máquinas. Cuando consiguieron uno,
     puede iniciar el proceso de lavado y esperar a que termine (lo que
     toma `` washtime`` minutos)."

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """The washing processes. It takes a ``car`` processes and tries
        to clean it."""
        yield self.env.timeout(WASHTIME)
        print("El lavadero elimino el %d%% de la suciedad del %s." %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """El proceso del automóvil (cada automóvil tiene un` `nombre``) llega al lavado de autos
     (`` cw``) y solicita una máquina de limpieza.

          Luego comienza el proceso de lavado, espera a que termine y
          deja para nunca volver ...
    """
    print('%s llega al lavadero a los %.2f minutos.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s entra al lavadero a los %.2f minutos.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s sale del lavadero a los %.2f minutos.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Cree un lavado de autos, una serie de autos iniciales y siga creando autos
     aprox. cada `` t_inter`` minutos. """
    # Create the carwash
    carwash = Carwash(env, num_machines, washtime)

    # Create 4 initial cars
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))


# Setup and start the simulation
print('Carwash')

random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute!
env.run(until=SIM_TIME)

Carwash
Carro 0 llega al lavadero a los 0.00 minutos.
Carro 1 llega al lavadero a los 0.00 minutos.
Carro 2 llega al lavadero a los 0.00 minutos.
Carro 3 llega al lavadero a los 0.00 minutos.
Carro 0 entra al lavadero a los 0.00 minutos.
Carro 1 entra al lavadero a los 0.00 minutos.
Carro 4 llega al lavadero a los 5.00 minutos.
El lavadero elimino el 97% de la suciedad del Carro 0.
El lavadero elimino el 67% de la suciedad del Carro 1.
Carro 0 sale del lavadero a los 5.00 minutos.
Carro 1 sale del lavadero a los 5.00 minutos.
Carro 2 entra al lavadero a los 5.00 minutos.
Carro 3 entra al lavadero a los 5.00 minutos.
Carro 5 llega al lavadero a los 10.00 minutos.
El lavadero elimino el 64% de la suciedad del Carro 2.
El lavadero elimino el 58% de la suciedad del Carro 3.
Carro 2 sale del lavadero a los 10.00 minutos.
Carro 3 sale del lavadero a los 10.00 minutos.
Carro 4 entra al lavadero a los 10.00 minutos.
Carro 5 entra al lavadero a los 10.00 minutos.
El lavadero elimino el 97% de l

In [11]:
RANDOM_SEED = 42
PT_MEAN = 10.0         # Avg. processing time in minutes
PT_SIGMA = 2.0         # Sigma of processing time
MTTF = 300.0           # Mean time to failure in minutes
BREAK_MEAN = 1 / MTTF  # Param. for expovariate distribution
REPAIR_TIME = 30.0     # Time it takes to repair a machine in minutes
JOB_DURATION = 30.0    # Duration of other jobs in minutes
NUM_MACHINES = 10      # Number of machines in the machine shop
WEEKS = 4              # Simulation time in weeks
SIM_TIME = WEEKS * 7 * 24 * 60  # Simulation time in minutes


def time_per_part():
    """Devolver el tiempo de procesamiento real para una parte concreta"""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Tiempo de vuelta hasta el siguiente fallo de una máquina."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """Una máquina produce partes y mi se rompe de vez en cuando.

     Si se rompe, solicita un * reparador * y continúa la producción.
     después de que se repare.

     Una máquina tiene un * nombre * y un número de * partes_hechas * hasta ahora.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Start "working" and "break_machine" processes for this machine.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce partes siempre que la simulación se ejecute.

         Al hacer una parte, la máquina puede romperse varias veces.
         Solicite un reparador cuando esto suceda.

        """
        while True:
            # Start making a new part
            done_in = time_per_part()
            while done_in:
                try:
                    # Working on the part
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Set to 0 to exit while loop.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # How much time left?

                    # Request a repairman. This will preempt its "other_job".
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """Rompe la máquina de vez en cuando."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Only break the machine if it is currently working.
                self.process.interrupt()


def other_jobs(env, repairman):
    """El otro trabajo (sin importancia) del reparador."""
    while True:
        # Start a new job
        done_in = JOB_DURATION
        while done_in:
            # Retry the job until it is done.
            # It's priority is lower than that of machine repairs.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start


# Prepara y empieza la simulación
print('Machine shop')
random.seed(RANDOM_SEED)

# Crea un ambiente y empiza el proceso de preparación
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Máquina %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Ejecuta!
env.run(until=SIM_TIME)

# Analisis/Resultados
print('Resultados del taller después de %s semanas:' % WEEKS)
for machine in machines:
    print('La %s hizo %d partes.' % (machine.name, machine.parts_made))

Machine shop
Resultados del taller después de 4 semanas:
La Máquina 0 hizo 3251 partes.
La Máquina 1 hizo 3273 partes.
La Máquina 2 hizo 3242 partes.
La Máquina 3 hizo 3343 partes.
La Máquina 4 hizo 3387 partes.
La Máquina 5 hizo 3244 partes.
La Máquina 6 hizo 3269 partes.
La Máquina 7 hizo 3185 partes.
La Máquina 8 hizo 3302 partes.
La Máquina 9 hizo 3279 partes.


In [12]:
import collections
import random

import simpy


RANDOM_SEED = 42
TICKETS = 50  # Number of tickets per movie
SIM_TIME = 120  # Simulate until


def moviegoer(env, movie, num_tickets, theater):
    """Un espectador lo intenta por una cantidad de tickets (* num_tickets *) para
     una cierta * película * en un * teatro *.

     Si la película se agota, ella abandona el teatro. Si ella consigue
     Al mostrador, ella trata de comprar un número de entradas. Si no es suficiente
     Quedan entradas, ella discute con el cajero y se va.

     Si a lo sumo queda un boleto después de que el espectador la compró
     entradas, el evento * agotado * para esta película se activa causando
     todos los espectadores restantes para salir.

    """
    with theater.counter.request() as my_turn:
        # Wait until its our turn or until the movie is sold out
        result = yield my_turn | theater.sold_out[movie]

        # Check if it's our turn of if movie is sold out
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            env.exit()

        # Check if enough tickets left.
        if theater.available[movie] < num_tickets:
            # Moviegoer leaves after some discussion
            yield env.timeout(0.5)
            env.exit()

        # Buy tickets
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            # Trigger the "sold out" event for the movie
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


def customer_arrivals(env, theater):
    """Crea nuevos * espectadores * hasta que el tiempo de simulación llegue a 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')


# Setup and start the simulation
print('Movie renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Create movie theater
counter = simpy.Resource(env, capacity=1)
movies = ['El Resplandor', 'La Monja', 'Los vengadores']
available = {movie: TICKETS for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

# Start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analysis/results
for movie in movies:
    if theater.sold_out[movie]:
        print('La pelicula "%s" se agotó %.1f minutos después de que el mostrador abriera.' % (movie, theater.when_sold_out[movie]))
        print('  Cantidad de personas que se fueron cuando se agotó la pelicula: %s' %
              theater.num_renegers[movie])

Movie renege
La pelicula "El Resplandor" se agotó 38.0 minutos después de que el mostrador abriera.
  Cantidad de personas que se fueron cuando se agotó la pelicula: 16
La pelicula "La Monja" se agotó 43.0 minutos después de que el mostrador abriera.
  Cantidad de personas que se fueron cuando se agotó la pelicula: 5
La pelicula "Los vengadores" se agotó 28.0 minutos después de que el mostrador abriera.
  Cantidad de personas que se fueron cuando se agotó la pelicula: 5


In [13]:
import itertools

RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # liters
THRESHOLD = 10             # Threshold for calling the tank truck (in %)
FUEL_TANK_SIZE = 50        # liters
FUEL_TANK_LEVEL = [5, 25]  # Min/max levels of fuel tanks (in liters)
REFUELING_SPEED = 2        # liters / second
TANK_TRUCK_TIME = 300      # Seconds it takes the tank truck to arrive
T_INTER = [30, 300]        # Create a car every [min, max] seconds
SIM_TIME = 1000            # Simulation time in seconds


def car(name, env, gas_station, fuel_pump):
    """Llega un coche a la gasolinera para tanquear.

     Solicita una de las bombas de combustible de la estación de servicio e intenta obtener el
     cantidad deseada de gas de ella. Si el embalse de las estaciones es
     agotado, el coche tiene que esperar a que llegue el camión cisterna.

    """
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s llegando a la gasolinera en %.1f' % (name, env.now))
    with gas_station.request() as req:
        start = env.now
        # Request one of the gas pumps
        yield req

        # Get the required amount of fuel
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)

        # The "actual" refueling process takes some time
        yield env.timeout(liters_required / REFUELING_SPEED)

        print('%s termina de tanquear en %.1f segundos.' % (name,
                                                          env.now - start))


def gas_station_control(env, fuel_pump):
    """Verifique periódicamente el nivel de la * bomba_de_combustible * y llame al tanque
     camión si el nivel cae por debajo de un umbral."""
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # We need to call the tank truck now!
            print('Calling tank truck at %d' % env.now)
            # Wait for the tank truck to arrive and refuel the station
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # Check every 10 seconds


def tank_truck(env, fuel_pump):
    """Llega a la estación de servicio después de un cierto retraso y lo reabastece.."""
    yield env.timeout(TANK_TRUCK_TIME)
    print('Camión cisterna llegando a la hora %d' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Camión cisterna reabasteciendo % .1f litros.' % ammount)
    yield fuel_pump.put(ammount)


def car_generator(env, gas_station, fuel_pump):
    """Generar autos nuevos que llegan a la estación de servicio.."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Car %d' % i, env, gas_station, fuel_pump))


# Setup and start the simulation
print('Gas Station refuelling')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)

Gas Station refuelling
Car 0 llegando a la gasolinera en 87.0
Car 0 termina de tanquear en 18.5 segundos.
Car 1 llegando a la gasolinera en 129.0
Car 1 termina de tanquear en 19.0 segundos.
Car 2 llegando a la gasolinera en 284.0
Car 2 termina de tanquear en 21.0 segundos.
Car 3 llegando a la gasolinera en 385.0
Car 3 termina de tanquear en 13.5 segundos.
Car 4 llegando a la gasolinera en 459.0
Calling tank truck at 460
Car 4 termina de tanquear en 22.0 segundos.
Car 5 llegando a la gasolinera en 705.0
Car 6 llegando a la gasolinera en 750.0
Camión cisterna llegando a la hora 760
Camión cisterna reabasteciendo  188.0 litros.
Car 6 termina de tanquear en 29.0 segundos.
Car 5 termina de tanquear en 76.5 segundos.
Car 7 llegando a la gasolinera en 891.0
Car 7 termina de tanquear en 13.0 segundos.


In [13]:
RANDOM_SEED = 42
SIM_TIME = 100


class BroadcastPipe(object):
    """Una canalización de difusión que permite que un proceso envíe mensajes a muchos.

     Esta construcción es útil cuando los consumidores de mensajes se están ejecutando en
     Diferentes tarifas que los generadores de mensajes y proporciona un evento.
     buffering a los procesos consumidores.

     Los parámetros se utilizan para crear una nueva.
    :class:`~simpy.resources.store.Store` instancia cada vez
    :meth:`get_output_conn()` es llamado.

    """
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Transmita un * valor * a todos los receptores."""
        if not self.pipes:
            raise RuntimeError('No hay tuberías de salida..')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        """Obtenga una nueva conexión de salida para esta tubería de difusión.

         El valor de retorno es a: class: `~ simpy.resources.store.Store`.

        """
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe


def message_generator(name, env, out_pipe):
    """A process which randomly generates messages."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        # messages are time stamped to later check if the consumer was
        # late getting them.  Note, using event.triggered to do this may
        # result in failure due to FIFO nature of simulation yields.
        # (i.e. if at the same env.now, message_generator puts a message
        # in the pipe first and then message_consumer gets from pipe,
        # the event.triggered will be True in the other order it will be
        # False
        msg = (env.now, '%s dice hola en %d' % (name, env.now))
        out_pipe.put(msg)


def message_consumer(name, env, in_pipe):
    """A process which consumes messages."""
    while True:
        # Get event for message pipe
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            # if message was already put into pipe, then
            # message_consumer was late getting to it. Depending on what
            # is being modeled this, may, or may not have some
            # significance
            print('LLega mensaje TARDE: en la hora %d: %s mensaje recibido: %s' %
                  (env.now, name, msg[1]))

        else:
            # message_consumer is synchronized with message_generator
            print('en la hora %d: %s mensaje recibido: %s.' %
                  (env.now, name, msg[1]))

        # Process does some other work, which may result in missing messages
        yield env.timeout(random.randint(4, 8))


# Setup and start the simulation
print('Process communication')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# For one-to-one or many-to-one type pipes, use Store
pipe = simpy.Store(env)
env.process(message_generator('Generator A', env, pipe))
env.process(message_consumer('Consumer A', env, pipe))

print('\nComunicacion 1 a 1\n')
env.run(until=SIM_TIME)

# For one-to many use BroadcastPipe
# (Note: could also be used for one-to-one,many-to-one or many-to-many)
env = simpy.Environment()
bc_pipe = BroadcastPipe(env)

env.process(message_generator('Generador A', env, bc_pipe))
env.process(message_consumer('Consumidor A', env, bc_pipe.get_output_conn()))
env.process(message_consumer('Consumidor B', env, bc_pipe.get_output_conn()))

print('\nComunicacion 1 a varios\n')
env.run(until=SIM_TIME)

Process communication

Comunicacion 1 a 1

en la hora 6: Consumer A mensaje recibido: Generator A dice hola en 6.
en la hora 12: Consumer A mensaje recibido: Generator A dice hola en 12.
en la hora 19: Consumer A mensaje recibido: Generator A dice hola en 19.
en la hora 26: Consumer A mensaje recibido: Generator A dice hola en 26.
en la hora 36: Consumer A mensaje recibido: Generator A dice hola en 36.
en la hora 46: Consumer A mensaje recibido: Generator A dice hola en 46.
en la hora 52: Consumer A mensaje recibido: Generator A dice hola en 52.
en la hora 58: Consumer A mensaje recibido: Generator A dice hola en 58.
LLega mensaje TARDE: en la hora 66: Consumer A mensaje recibido: Generator A dice hola en 65
en la hora 75: Consumer A mensaje recibido: Generator A dice hola en 75.
en la hora 85: Consumer A mensaje recibido: Generator A dice hola en 85.
en la hora 95: Consumer A mensaje recibido: Generator A dice hola en 95.

Comunicacion 1 a varios

en la hora 10: Consumidor A mensaje r

In [14]:
SIM_DURATION = 100


class Cable(object):
    """Esta clase representa la propagación a través de un cable."""
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latency(self, value):
        yield self.env.timeout(self.delay)
        self.store.put(value)

    def put(self, value):
        self.env.process(self.latency(value))

    def get(self):
        return self.store.get()


def sender(env, cable):
    """Un proceso que genera mensajes al azar."""
    while True:
        # wait for next transmission
        yield env.timeout(5)
        cable.put('El remitente envió esto a %d' % env.now)


def receiver(env, cable):
    """Un proceso que consume mensajes."""
    while True:
        # Get event for message pipe
        msg = yield cable.get()
        print('Recibido este en %d mientras %s' % (env.now, msg))


# Setup and start the simulation
print('Event Latency')
env = simpy.Environment()

cable = Cable(env, 10)
env.process(sender(env, cable))
env.process(receiver(env, cable))

env.run(until=SIM_DURATION)

Event Latency
Recibido este en 15 mientras El remitente envió esto a 5
Recibido este en 20 mientras El remitente envió esto a 10
Recibido este en 25 mientras El remitente envió esto a 15
Recibido este en 30 mientras El remitente envió esto a 20
Recibido este en 35 mientras El remitente envió esto a 25
Recibido este en 40 mientras El remitente envió esto a 30
Recibido este en 45 mientras El remitente envió esto a 35
Recibido este en 50 mientras El remitente envió esto a 40
Recibido este en 55 mientras El remitente envió esto a 45
Recibido este en 60 mientras El remitente envió esto a 50
Recibido este en 65 mientras El remitente envió esto a 55
Recibido este en 70 mientras El remitente envió esto a 60
Recibido este en 75 mientras El remitente envió esto a 65
Recibido este en 80 mientras El remitente envió esto a 70
Recibido este en 85 mientras El remitente envió esto a 75
Recibido este en 90 mientras El remitente envió esto a 80
Recibido este en 95 mientras El remitente envió esto a 85
